In [48]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np

df = pd.read_csv('ds_salaries.csv')
df.drop(df.columns[8], axis = 1, inplace = True)


In [49]:
def create_sequence(data, seq_length):
    xlist = []
    ylist = []
    for i in range(len(data)-seq_length):
        seq = data[i:i+seq_length]
        xlist.append(seq)
        ylist.append(data[i+seq_length])
    x_arr = np.array(xlist)
    y_arr = np.array(ylist)
    return x_arr, y_arr

In [50]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,CA,M


In [51]:
df.drop(df.columns[5], axis = 1, inplace = True)
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_in_usd,employee_residence,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,85847,ES,ES,L
1,2023,MI,CT,ML Engineer,30000,30000,US,US,S
2,2023,MI,CT,ML Engineer,25500,25500,US,US,S
3,2023,SE,FT,Data Scientist,175000,175000,CA,CA,M
4,2023,SE,FT,Data Scientist,120000,120000,CA,CA,M


In [52]:
df.drop(df.columns[8], axis = 1, inplace = True)
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_in_usd,employee_residence,company_location
0,2023,SE,FT,Principal Data Scientist,80000,85847,ES,ES
1,2023,MI,CT,ML Engineer,30000,30000,US,US
2,2023,MI,CT,ML Engineer,25500,25500,US,US
3,2023,SE,FT,Data Scientist,175000,175000,CA,CA
4,2023,SE,FT,Data Scientist,120000,120000,CA,CA


In [53]:
df.drop(df.columns[3], axis = 1, inplace = True)
df.head()

,work_year,experience_level,employment_type,salary,salary_in_usd,employee_residence,company_location
0,2023,SE,FT,80000,85847,ES,ES
1,2023,MI,CT,30000,30000,US,US
2,2023,MI,CT,25500,25500,US,US
3,2023,SE,FT,175000,175000,CA,CA
4,2023,SE,FT,120000,120000,CA,CA


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           3755 non-null   int64 
 1   experience_level    3755 non-null   object
 2   employment_type     3755 non-null   object
 3   salary              3755 non-null   int64 
 4   salary_in_usd       3755 non-null   int64 
 5   employee_residence  3755 non-null   object
 6   company_location    3755 non-null   object
dtypes: int64(3), object(4)
memory usage: 205.5+ KB


In [55]:
df["experience_level"].value_counts()

experience_level
SE    2516
MI     805
EN     320
EX     114
Name: count, dtype: int64

In [56]:
normalizando_experencia = {
    "EN":1,
    "MI":2,
    "SE":3,
    "EX":4
}

df['experience_level'] = df['experience_level'].map(normalizando_experencia)

df.head()

,work_year,experience_level,employment_type,salary,salary_in_usd,employee_residence,company_location
0,2023,3,FT,80000,85847,ES,ES
1,2023,2,CT,30000,30000,US,US
2,2023,2,CT,25500,25500,US,US
3,2023,3,FT,175000,175000,CA,CA
4,2023,3,FT,120000,120000,CA,CA


In [57]:
df.drop(df.columns[5], axis = 1, inplace = True)
df.head()

,work_year,experience_level,employment_type,salary,salary_in_usd,company_location
0,2023,3,FT,80000,85847,ES
1,2023,2,CT,30000,30000,US
2,2023,2,CT,25500,25500,US
3,2023,3,FT,175000,175000,CA
4,2023,3,FT,120000,120000,CA


In [58]:
df.drop(df.columns[5], axis = 1, inplace = True)
df.head()

,work_year,experience_level,employment_type,salary,salary_in_usd
0,2023,3,FT,80000,85847
1,2023,2,CT,30000,30000
2,2023,2,CT,25500,25500
3,2023,3,FT,175000,175000
4,2023,3,FT,120000,120000


In [59]:
normalizando_tipo_emprego = {
    "FT":1,
    "PT":2,
    "CT":3,
    "FL":4
}

df['employment_type'] = df['employment_type'].map(normalizando_tipo_emprego)

In [60]:
df["employment_type"].value_counts()

employment_type
1    3718
2      17
3      10
4      10
Name: count, dtype: int64

In [61]:
df.head()

,work_year,experience_level,employment_type,salary,salary_in_usd
0,2023,3,1,80000,85847
1,2023,2,3,30000,30000
2,2023,2,3,25500,25500
3,2023,3,1,175000,175000
4,2023,3,1,120000,120000


In [62]:
scaler = MinMaxScaler()
# df_normalizado = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)



In [63]:
X = df.iloc[:,:-1].values
y = df.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Definir a arquitetura do MLP
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
])

# Compilar o modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

# Treinar o modelo
model.fit(X_train, y_train, epochs=100, batch_size=36, validation_data=(X_test, y_test))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                320       
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
Total params: 4,480
Trainable params: 4,480
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
84/84 [==============================] - 1s 4ms/step - loss: 22693675008.0000 - accuracy: 0.0000e+00 - val_loss: 29256202240.0000 - val_accuracy: 0.0000e+00
Epoch 2/100
84/84 [==============================] - 0s 3ms/step - loss: 21846024192.0000 - accuracy: 0.0000e+00 - val_loss: 27202795520.0000 - val_accuracy: 0.0000e+00
Epoch 3/100
84/84 [==============================] - 0s 3ms/step - loss: 21780377600.0000 - accuracy: 0.0000e+